<H1>Load Datasets<H1>

In [2]:
import pandas as pd
import os

path = os.getcwd()

pd.set_option('display.max_columns', 200)

bracket_training = pd.read_csv(f"{path}/bracket_training.csv", sep= ",")
bracket_training

,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion
0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,164,29,317,694,164,694,694
1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,164,51,334,328,164,334,164
2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,164,51,288,559,164,559,559
3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 10:28:56 -0400,311,610,490,559,311,490,490
4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,2022-03-16,2828017,2024-03-20 20:14:52 -0400,37,457,387,169,457,387,457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129997,70315,720.0,80130,39.5414,-104.9218,751.0,DENVER,2023-05-01,3602437,2024-03-21 11:53:31 -0400,311,457,334,694,311,694,311
129998,51130,832.0,77407,29.6625,-95.7272,618.0,HOUSTON,2024-07-17,2260739,2024-03-19 23:06:28 -0400,311,8,288,694,8,288,288
129999,36902,NaN,89503,39.5354,-119.8374,811.0,RENO,2021-02-20,1298238,2024-03-18 16:17:25 -0400,164,457,288,169,164,288,164
130000,57171,205.0,35244,33.3538,-86.8254,630.0,BIRMINGHAM (ANN & TUSC),2019-06-01,681594,2024-03-17 21:14:21 -0400,164,8,334,559,164,559,559


In [3]:

college_info = pd.read_csv(f'{path}/institutions.csv', sep = ',', encoding= 'utf-8')

#college_info['InstitutionID'].nunique()

college_info_id = college_info.set_index('InstitutionID')

college_info_id
# college_info.sort_index( ascending=True)

,InstitutionName,InstitutionNickname,InstitutionAbbreviation,InstitutionCity,InstitutionState,InstitutionPostalCode,InstitutionDMACode,InstitutionDMADescription,InstitutionLatitude,InstitutionLongitude,InstitutionConference,InstitutionEnrollment_Male,InstitutionEnrollment_Female,InstitutionEnrollment_Total,InstitutionNCAAMemberSinceDate,RegularSeasonWins,RegularSeasonLosses,RegularSeasonAverageAttendance,RegularSeasonAverageScore
InstitutionID,,,,,,,,,,,,,,,,,,,
626,San Diego St.,Aztecs,SDSU,San Diego,CA,92182,825,SAN DIEGO,32.77525,-117.07120,Mountain West Conference,11992,16456,28448,09/01/1947,24,10,9832.94,74.59
402,McNeese,Cowboys,MCNEES,Lake Charles,LA,70609,643,LAKE CHARLES,30.17819,-93.21722,Southland Conference,2245,3462,5707,09/01/1965,30,3,3085.58,80.42
648,South Carolina,Gamecocks,SO CAR,Columbia,SC,29208,546,"COLUMBIA, SC",33.99679,-81.02693,Southeastern Conference,11273,14774,26047,09/01/1936,26,7,11865.45,72.03
175,Dayton,Flyers,DAY,Dayton,OH,45469-1230,542,DAYTON,39.73955,-84.17611,Atlantic 10 Conference,4245,3872,8117,09/01/1951,24,8,9187.23,74.74
317,James Madison,Dukes,JMU,Harrisonburg,VA,22807,569,HARRISONBURG,38.43631,-78.87048,Sun Belt Conference,8342,12004,20346,09/01/1969,31,3,4471.35,84.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
457,North Carolina,Tar Heels,UNC,Chapel Hill,NC,27514,560,RALEIGH - DURHAM (FAYETVLLE),35.91177,-79.05097,Atlantic Coast Conference,8068,12174,20242,09/01/1906,27,7,15767.47,81.47
731,Utah St.,Aggies,UT ST,Logan,UT,84322,770,SALT LAKE CITY,41.74075,-111.81390,Mountain West Conference,7236,8552,15788,09/01/1938,27,6,7027.67,79.79
416,Michigan St.,Spartans,MSU,East Lansing,MI,48824,551,LANSING,42.73212,-84.47611,Big Ten Conference,18557,19905,38462,09/01/1919,19,14,14005.79,73.12


In [4]:
'''Merge Vs Join'''

# test_df1 = pd.merge(left=bracket_training,
#                     how = "left",
#                     right = college_info_id.add_prefix("EW_"),
#                     left_on = "SemifinalWinner_East_West",
#                     right_index=True,
#                     )

# test_df2 = pd.merge(left= test_df1,
#                     right= college_info_id.add_prefix("SM_"),
#                     how = "left",
#                     left_on = "SemifinalWinner_South_Midwest",
#                     right_index=True
#                     )


test_df = bracket_training.join(
    college_info_id.add_prefix("EW_"), on="SemifinalWinner_East_West"
).join(
    college_info_id.add_prefix("SM_"), on="SemifinalWinner_South_Midwest"
)
# print(f'Number of Unique Values: {test_df1['CustomerID'].nunique()}')

test_df


,CustomerID,CustomerAreaCode,CustomerPostalCode,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,CustomerDMADescription,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion,EW_InstitutionName,EW_InstitutionNickname,EW_InstitutionAbbreviation,EW_InstitutionCity,EW_InstitutionState,EW_InstitutionPostalCode,EW_InstitutionDMACode,EW_InstitutionDMADescription,EW_InstitutionLatitude,EW_InstitutionLongitude,EW_InstitutionConference,EW_InstitutionEnrollment_Male,EW_InstitutionEnrollment_Female,EW_InstitutionEnrollment_Total,EW_InstitutionNCAAMemberSinceDate,EW_RegularSeasonWins,EW_RegularSeasonLosses,EW_RegularSeasonAverageAttendance,EW_RegularSeasonAverageScore,SM_InstitutionName,SM_InstitutionNickname,SM_InstitutionAbbreviation,SM_InstitutionCity,SM_InstitutionState,SM_InstitutionPostalCode,SM_InstitutionDMACode,SM_InstitutionDMADescription,SM_InstitutionLatitude,SM_InstitutionLongitude,SM_InstitutionConference,SM_InstitutionEnrollment_Male,SM_InstitutionEnrollment_Female,SM_InstitutionEnrollment_Total,SM_InstitutionNCAAMemberSinceDate,SM_RegularSeasonWins,SM_RegularSeasonLosses,SM_RegularSeasonAverageAttendance,SM_RegularSeasonAverageScore
0,47028,NaN,36093,32.5622,-86.0994,698.0,MONTGOMERY (SELMA),2021-12-25,1723503,2024-03-19 10:27:15 -0400,164,29,317,694,164,694,694,UConn,Huskies,UCONN,Storrs,CT,06269,533,HARTFORD & NEW HAVEN,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Tennessee,Volunteers,TENN,Knoxville,TN,37996,557,KNOXVILLE,35.95208,-83.92585,Southeastern Conference,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
1,3511,616.0,49464,42.8256,-86.0104,563.0,GRAND RAPIDS - KALMZOO - B. CRK,2021-04-02,963479,2024-03-18 10:16:39 -0400,164,51,334,328,164,334,164,UConn,Huskies,UCONN,Storrs,CT,06269,533,HARTFORD & NEW HAVEN,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Kentucky,Wildcats,UK,Lexington,KY,40506,541,LEXINGTON,38.03891,-84.50475,Southeastern Conference,9596,13127,22723,09/01/1936,23,9,17427.94,89.44
2,58445,703.0,22210,38.8808,-77.1129,511.0,"WASHINGTON, DC (HAGRSTWN)",2021-04-02,810038,2024-03-18 00:21:47 -0400,164,51,288,559,164,559,559,UConn,Huskies,UCONN,Storrs,CT,06269,533,HARTFORD & NEW HAVEN,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Purdue,Boilermakers,PUR,West Lafayette,IN,47907-2040,582,"LAFAYETTE, IN",40.42821,-86.91444,Big Ten Conference,21670,16279,37949,09/01/1914,29,4,13329.06,83.39
3,28833,NaN,78218,29.4969,-98.4032,641.0,SAN ANTONIO,2023-11-16,3384825,2024-03-21 10:28:56 -0400,311,610,490,559,311,490,490,Iowa St.,Cyclones,IA ST,Ames,IA,50011,679,DES MOINES - AMES,42.02621,-93.64851,Big 12 Conference,14070,11171,25241,09/01/1908,27,7,12059.44,75.56,NC State,Wolfpack,NC ST,Raleigh,NC,27695,560,RALEIGH - DURHAM (FAYETVLLE),35.78511,-78.67451,Atlantic Coast Conference,11649,11794,23443,09/01/1941,22,14,12147.11,76.36
4,37899,NaN,14212,42.8946,-78.8245,514.0,BUFFALO,2022-03-16,2828017,2024-03-20 20:14:52 -0400,37,457,387,169,457,387,457,North Carolina,Tar Heels,UNC,Chapel Hill,NC,27514,560,RALEIGH - DURHAM (FAYETVLLE),35.91177,-79.05097,Atlantic Coast Conference,8068,12174,20242,09/01/1906,27,7,15767.47,81.47,Marquette,Golden Eagles,MARQ,Milwaukee,WI,53233,617,MILWAUKEE,43.03903,-87.92796,Big East Conference,3328,4200,7528,09/01/1928,25,9,14084.65,78.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129997,70315,720.0,80130,39.5414,-104.9218,751.0,DENVER,2023-05-01,3602437,2024-03-21 11:53:31 -0400,311,457,334,694,311,694,311,Iowa St.,Cyclones,IA ST,Ames,IA,50011,679,DES MOINES - AMES,42.02621,-93.64851,Big 12 Conference,14070,1

In [9]:
'''Deleting Unwanted Columns'''

print(test_df.columns)

classic1_df = test_df[
    [
        'CustomerID',
        #'CustomerAreaCode', 'CustomerPostalCode',
       'CustomerPostalCodeLatitude', 'CustomerPostalCodeLongitude',
       'CustomerDMACode',
        #'CustomerDMADescription',
       'NCAACustomerRecordCreated', 'BracketEntryId',
       'BracketEntryCreatedDate', 'RegionWinner_East', 'RegionWinner_West',
       'RegionWinner_South', 'RegionWinner_Midwest',
       'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest',
       'NationalChampion', 'EW_InstitutionName',
        # 'EW_InstitutionNickname',
        #'EW_InstitutionAbbreviation', 'EW_InstitutionCity',
        #'EW_InstitutionState', 'EW_InstitutionPostalCode',
       'EW_InstitutionDMACode',
        # 'EW_InstitutionDMADescription',
       'EW_InstitutionLatitude', 'EW_InstitutionLongitude',
       'EW_InstitutionConference', 'EW_InstitutionEnrollment_Male',
       'EW_InstitutionEnrollment_Female', 'EW_InstitutionEnrollment_Total',
       'EW_InstitutionNCAAMemberSinceDate', 'EW_RegularSeasonWins',
       'EW_RegularSeasonLosses', 'EW_RegularSeasonAverageAttendance',
       'EW_RegularSeasonAverageScore', 'SM_InstitutionName',
        #'SM_InstitutionNickname', 'SM_InstitutionAbbreviation',
        #'SM_InstitutionCity', 'SM_InstitutionState', 'SM_InstitutionPostalCode',
       'SM_InstitutionDMACode',
        #'SM_InstitutionDMADescription',
       'SM_InstitutionLatitude', 'SM_InstitutionLongitude',
        #'SM_InstitutionConference',
       'SM_InstitutionEnrollment_Male',
       'SM_InstitutionEnrollment_Female', 'SM_InstitutionEnrollment_Total',
       'SM_InstitutionNCAAMemberSinceDate', 'SM_RegularSeasonWins',
       'SM_RegularSeasonLosses', 'SM_RegularSeasonAverageAttendance',
       'SM_RegularSeasonAverageScore'
    ]
]

classic1_df

Index(['CustomerID', 'CustomerAreaCode', 'CustomerPostalCode',
       'CustomerPostalCodeLatitude', 'CustomerPostalCodeLongitude',
       'CustomerDMACode', 'CustomerDMADescription',
       'NCAACustomerRecordCreated', 'BracketEntryId',
       'BracketEntryCreatedDate', 'RegionWinner_East', 'RegionWinner_West',
       'RegionWinner_South', 'RegionWinner_Midwest',
       'SemifinalWinner_East_West', 'SemifinalWinner_South_Midwest',
       'NationalChampion', 'EW_InstitutionName', 'EW_InstitutionNickname',
       'EW_InstitutionAbbreviation', 'EW_InstitutionCity',
       'EW_InstitutionState', 'EW_InstitutionPostalCode',
       'EW_InstitutionDMACode', 'EW_InstitutionDMADescription',
       'EW_InstitutionLatitude', 'EW_InstitutionLongitude',
       'EW_InstitutionConference', 'EW_InstitutionEnrollment_Male',
       'EW_InstitutionEnrollment_Female', 'EW_InstitutionEnrollment_Total',
       'EW_InstitutionNCAAMemberSinceDate', 'EW_RegularSeasonWins',
       'EW_RegularSeasonLosses', 'EW_

,CustomerID,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion,EW_InstitutionName,EW_InstitutionDMACode,EW_InstitutionLatitude,EW_InstitutionLongitude,EW_InstitutionConference,EW_InstitutionEnrollment_Male,EW_InstitutionEnrollment_Female,EW_InstitutionEnrollment_Total,EW_InstitutionNCAAMemberSinceDate,EW_RegularSeasonWins,EW_RegularSeasonLosses,EW_RegularSeasonAverageAttendance,EW_RegularSeasonAverageScore,SM_InstitutionName,SM_InstitutionDMACode,SM_InstitutionLatitude,SM_InstitutionLongitude,SM_InstitutionEnrollment_Male,SM_InstitutionEnrollment_Female,SM_InstitutionEnrollment_Total,SM_InstitutionNCAAMemberSinceDate,SM_RegularSeasonWins,SM_RegularSeasonLosses,SM_RegularSeasonAverageAttendance,SM_RegularSeasonAverageScore
0,47028,32.5622,-86.0994,698.0,2021-12-25,1723503,2024-03-19 10:27:15 -0400,164,29,317,694,164,694,694,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Tennessee,557,35.95208,-83.92585,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
1,3511,42.8256,-86.0104,563.0,2021-04-02,963479,2024-03-18 10:16:39 -0400,164,51,334,328,164,334,164,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Kentucky,541,38.03891,-84.50475,9596,13127,22723,09/01/1936,23,9,17427.94,89.44
2,58445,38.8808,-77.1129,511.0,2021-04-02,810038,2024-03-18 00:21:47 -0400,164,51,288,559,164,559,559,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Purdue,582,40.42821,-86.91444,21670,16279,37949,09/01/1914,29,4,13329.06,83.39
3,28833,29.4969,-98.4032,641.0,2023-11-16,3384825,2024-03-21 10:28:56 -0400,311,610,490,559,311,490,490,Iowa St.,679,42.02621,-93.64851,Big 12 Conference,14070,11171,25241,09/01/1908,27,7,12059.44,75.56,NC State,560,35.78511,-78.67451,11649,11794,23443,09/01/1941,22,14,12147.11,76.36
4,37899,42.8946,-78.8245,514.0,2022-03-16,2828017,2024-03-20 20:14:52 -0400,37,457,387,169,457,387,457,North Carolina,560,35.91177,-79.05097,Atlantic Coast Conference,8068,12174,20242,09/01/1906,27,7,15767.47,81.47,Marquette,617,43.03903,-87.92796,3328,4200,7528,09/01/1928,25,9,14084.65,78.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129997,70315,39.5414,-104.9218,751.0,2023-05-01,3602437,2024-03-21 11:53:31 -0400,311,457,334,694,311,694,311,Iowa St.,679,42.02621,-93.64851,Big 12 Conference,14070,11171,25241,09/01/1908,27,7,12059.44,75.56,Tennessee,557,35.95208,-83.92585,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
129998,51130,29.6625,-95.7272,618.0,2024-07-17,2260739,2024-03-19 23:06:28 -0400,311,8,288,694,8,288,288,Alabama,630,33.21188,-87.54597,Southeastern Conference,14152,18306,32458,09/01/1936,21,11,10947.59,90.75,Houston,618,29.72039,-95.34354,18290,19653,37943,09/01/1949,30,4,9347.35,73.03
129999,36902,39.5354,-119.8374,811.0,2021-02-20,1298238,2024-03-18 16:17:25 -0400,164,457,288,169,164,288,164,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Houston,618,29.72039,-95.34354,18290,19653,37943,09/01/1949,30,4,9347.35,73.03
130000,57171,33.3538,-86.8254,630.0,2019-06-01,681594,2024-03-17 21:14:21 -0400,164,8,334,559,164,559,559,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Purdue,582,40.42821,-86.91444,21670,16279,37949,09/01/1914,29,4,13329.06,83.39


In [16]:
'''Checking NAs'''

#Since the Nan values surmount to around 0.5% of the sample size, I am choosing to get rid of them
# for column in classic1_df.columns:
#     print(f'{column}: {round(((classic1_df[column].isna().sum())/classic1_df.shape[0]) * 100, 3)}  %Nan Values')

classic1_df.dropna(axis = 0)

,CustomerID,CustomerPostalCodeLatitude,CustomerPostalCodeLongitude,CustomerDMACode,NCAACustomerRecordCreated,BracketEntryId,BracketEntryCreatedDate,RegionWinner_East,RegionWinner_West,RegionWinner_South,RegionWinner_Midwest,SemifinalWinner_East_West,SemifinalWinner_South_Midwest,NationalChampion,EW_InstitutionName,EW_InstitutionDMACode,EW_InstitutionLatitude,EW_InstitutionLongitude,EW_InstitutionConference,EW_InstitutionEnrollment_Male,EW_InstitutionEnrollment_Female,EW_InstitutionEnrollment_Total,EW_InstitutionNCAAMemberSinceDate,EW_RegularSeasonWins,EW_RegularSeasonLosses,EW_RegularSeasonAverageAttendance,EW_RegularSeasonAverageScore,SM_InstitutionName,SM_InstitutionDMACode,SM_InstitutionLatitude,SM_InstitutionLongitude,SM_InstitutionEnrollment_Male,SM_InstitutionEnrollment_Female,SM_InstitutionEnrollment_Total,SM_InstitutionNCAAMemberSinceDate,SM_RegularSeasonWins,SM_RegularSeasonLosses,SM_RegularSeasonAverageAttendance,SM_RegularSeasonAverageScore
0,47028,32.5622,-86.0994,698.0,2021-12-25,1723503,2024-03-19 10:27:15 -0400,164,29,317,694,164,694,694,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Tennessee,557,35.95208,-83.92585,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
1,3511,42.8256,-86.0104,563.0,2021-04-02,963479,2024-03-18 10:16:39 -0400,164,51,334,328,164,334,164,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Kentucky,541,38.03891,-84.50475,9596,13127,22723,09/01/1936,23,9,17427.94,89.44
2,58445,38.8808,-77.1129,511.0,2021-04-02,810038,2024-03-18 00:21:47 -0400,164,51,288,559,164,559,559,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Purdue,582,40.42821,-86.91444,21670,16279,37949,09/01/1914,29,4,13329.06,83.39
3,28833,29.4969,-98.4032,641.0,2023-11-16,3384825,2024-03-21 10:28:56 -0400,311,610,490,559,311,490,490,Iowa St.,679,42.02621,-93.64851,Big 12 Conference,14070,11171,25241,09/01/1908,27,7,12059.44,75.56,NC State,560,35.78511,-78.67451,11649,11794,23443,09/01/1941,22,14,12147.11,76.36
4,37899,42.8946,-78.8245,514.0,2022-03-16,2828017,2024-03-20 20:14:52 -0400,37,457,387,169,457,387,457,North Carolina,560,35.91177,-79.05097,Atlantic Coast Conference,8068,12174,20242,09/01/1906,27,7,15767.47,81.47,Marquette,617,43.03903,-87.92796,3328,4200,7528,09/01/1928,25,9,14084.65,78.29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129996,80352,36.3425,-88.8554,632.0,2024-03-22,2437071,2024-03-20 10:57:35 -0400,311,51,235,694,51,694,51,Baylor,625,31.54687,-97.12104,Big 12 Conference,6008,8877,14885,09/01/1922,23,10,9517.79,80.55,Tennessee,557,35.95208,-83.92585,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
129997,70315,39.5414,-104.9218,751.0,2023-05-01,3602437,2024-03-21 11:53:31 -0400,311,457,334,694,311,694,311,Iowa St.,679,42.02621,-93.64851,Big 12 Conference,14070,11171,25241,09/01/1908,27,7,12059.44,75.56,Tennessee,557,35.95208,-83.92585,15269,18536,33805,09/01/1909,24,8,16065.47,79.47
129998,51130,29.6625,-95.7272,618.0,2024-07-17,2260739,2024-03-19 23:06:28 -0400,311,8,288,694,8,288,288,Alabama,630,33.21188,-87.54597,Southeastern Conference,14152,18306,32458,09/01/1936,21,11,10947.59,90.75,Houston,618,29.72039,-95.34354,18290,19653,37943,09/01/1949,30,4,9347.35,73.03
129999,36902,39.5354,-119.8374,811.0,2021-02-20,1298238,2024-03-18 16:17:25 -0400,164,457,288,169,164,288,164,UConn,533,41.80910,-72.24995,Big East Conference,10645,11834,22479,09/01/1910,31,3,14017.88,81.47,Houston,618,29.72039,-95.34354,18290,19653,37943,09/01/1949,30,4,9347.35,73.03


In [22]:
num_brackets_df = classic1_df.groupby(by = 'CustomerID').agg({'BracketEntryId': 'count'})
num_brackets_df[num_brackets_df['BracketEntryId'] > 1]

perc_Id_w_multiple_brackets = num_brackets_df[num_brackets_df['BracketEntryId'] > 1].shape[0] / num_brackets_df.shape[0]

print(perc_Id_w_multiple_brackets)


0.2714167444183082
